In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import glob

In [3]:
#majors2020 = pd.read_table("data/uc.berkeley.student.majors.2020.txt")
df = pd.DataFrame()
files = glob.glob("data/*.txt")
course_data = glob.glob("data/uc.berkeley.student.course.data.*.txt")
majors = glob.glob("data/uc.berkeley.student.majors.*.txt")
pathways = glob.glob("data/uc.berkeley.stem.*.txt")


In [10]:
course_data.sort()
student_courses = pd.DataFrame()
for f in course_data:
    student_courses = student_courses.append(pd.read_table(f))

In [33]:
student_courses = student_courses[~student_courses['ppsk'].isnull()]
student_courses = student_courses[student_courses['ppsk'] != -1]

TypeError: ufunc 'invert' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [32]:
student_courses

,year.name,ppsk,undergrad.status,course.department,course.subject,course.number,offering.type,section.number,ccn,room.share,...,Section Nbr,Course Control Nbr,Room Share Bundle Nbr,Course Title Nm,Instr Name Concat,Grade Subtype Desc,Grade Type Desc,Grade Nm,Grade Points Nbr,Student Credit Hrs Nbr
0,2007 Summer,-1.0,Graduate,Anthropology,Anthropology,3AC,Primary,1,12030,12030.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2007 Summer,-1.0,Graduate,Anthropology,Anthropology,136B,Primary,1,12120,12120.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2007 Summer,-1.0,Graduate,Buddhist Studies Grad Grp,Buddhist Studies,C50,Primary,1,18905,200018.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2007 Summer,-1.0,Graduate,Business,Business Admin-Undergrad,102A,Primary,5,20150,20150.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2007 Summer,-1.0,Graduate,Business,Business Admin-Undergrad,106,Primary,4,20235,20235.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
372773,2008 Summer,-1.0,Undergraduate,Statistics,Statistics,2,Secondary,103,83540,83540.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
372774,2008 Summer,-1.0,Undergraduate,Statistics,Statistics,20,Secondary,102,83560,83560.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
372775,2008 Summer,-1.0,Undergraduate,Statistics,Statistics,21,Secondary,101,83570,83570.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
372776,2008 Summer,-1.0,Undergraduate,UG Interdisciplinary Studies,American Studies,179AC,Secondary,101,11309,11309.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
print(np.sort(student_courses['course.department'].unique()))

['-' 'African American Studies' 'Ag & Env Chem Grad Grp'
 'Ag & Resource Econ & Pol' 'Anc Hist Med Arc Grad Grp' 'Anthropology'
 'Applied Sci & Tech Grad Grp' 'Architecture' 'Art Practice' 'Astronomy'
 'Bioengineering' 'Biophysics Grad Grp' 'Buddhist Studies Grad Grp'
 'Business' 'Chem & Biomolecular Eng' 'Chemistry'
 'City & Regional Planning' 'Civil & Environmental Eng' 'Classics'
 'College Writing Programs' 'Comparative Biochem Grad Grp'
 'Comparative Literature' 'Computational Biology Grad Grp'
 'Critical Theory Grad Grp' 'Data Science' 'Demography'
 'Design Innovation' 'Development Eng Grad Grp'
 'Development Practice Grad Grp' 'Earth & Planetary Science'
 'East Asian Lang & Culture' 'Economics' 'Education'
 'Electrical Eng & Computer Sci' 'Energy & Resources Grad Grp'
 'Engineering Science' 'English' 'Env Sci, Policy, & Mgmt'
 'Ethnic Studies' 'European Studies Grad Grp'
 'FPF-African American Studies' 'FPF-Anthropology' 'FPF-Art Practice'
 'FPF-Astronomy' 'FPF-Chemistry' 'FPF-Cl